In [ ]:
from sklearn.datasets import fetch_20newsgroups as getData
import numpy as np
import re

In [ ]:
X,y = getData(subset='train',remove=('headers','footers','quotes'),return_X_y=True)

In [ ]:
len(X)

In [ ]:
print(X[0])

In [ ]:
from collections import defaultdict

In [ ]:
fr = defaultdict(int)
for doc in X:
    for token in re.split('\W+',doc.lower()):
        fr[token] += 1

In [ ]:
cutOffValue = 10
processed_corpus = []
for doc in X:
    for token in re.split('\W+',doc.lower()):
        if fr[token] >= cutOffValue:
            processed_corpus.append(token)
        

In [ ]:
allWords = np.array(list(fr.keys()))
allCounts = np.array(list(fr.values()))
vocab = allWords[allCounts >= cutOffValue]
wordCounts = allCounts[allCounts >= cutOffValue]

In [ ]:
alpha = 0.75
wordCounts = wordCounts**alpha
probs = wordCounts/np.sum(wordCounts)

In [ ]:
numWords = len(vocab)
W2I = dict(zip(vocab,np.arange(numWords)))
I2W = dict(zip(np.arange(numWords),vocab))

In [ ]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

In [ ]:
def gradientStep(w,cPos,cNeg,lr=0.001):
    cPos_new = cPos - lr*(sigmoid(w.dot(cPos))-1)*w
    w_new = w - lr*(sigmoid(w.dot(cPos))-1)*cPos
    cNeg_new = np.zeros(cNeg.shape)
    for i in range(cNeg.shape[0]):
        v = sigmoid(cNeg[i,:].dot(w))
        w_new -= lr*v*cNeg[i,:]
        cNeg_new[i,:] = cNeg[i,:] - lr*v*w
    return w_new,cPos_new,cNeg_new

In [ ]:
def word2vecSGNS(W,C,doc,vocab,W2I,I2W,probs,winSize=2,k=4):
    numTokens = len(doc)
    for i in range(numTokens):
        for j in range(i-winSize,i+winSize):
            if j != i and j >= 0 and j < numTokens:
                wIdx = W2I[doc[i]]
                posIdx = W2I[doc[j]]
                w = W[wIdx,:]
                cPos = C[posIdx,:]
                cNeg = np.zeros((k,C.shape[1]))
                m = 0
                negIdx = []
                for negC in np.random.choice(list(vocab),k,list(probs)):
                    negIdx.append(W2I[negC])
                    cNeg[m,:] = C[W2I[negC],:]
                    m += 1
        w_new,cPos_new,cNeg_new = gradientStep(w,cPos,cNeg)
        W[wIdx,:] = w_new
        C[posIdx,:] = cPos_new
        C[negIdx,:] = cNeg_new
    return W,C

                

In [ ]:
W = np.random.rand(numWords,100)
C = np.random.rand(numWords,100)
W,C = word2vecSGNS(W,C,processed_corpus,vocab,W2I,I2W,probs)

In [ ]:
#E = W+C
#E = W